# 第4章 LangChainの基礎

## 設定

In [1]:
import json
import os
import time

from dotenv import load_dotenv
dotenv_path = "../.env"
load_dotenv(dotenv_path)

True

In [2]:
!pip install langchain-core==0.3.0
!pip install langchain-openai==0.2.0
!pip install langchain-community==0.3.0
!pip install GitPython==3.1.43
!pip install langchain-text-aplitters==0.3.0
!pip install langchain-chroma==0.1.4


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_core-0.3.29-py3-none-any.whl (411 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.0
    Uninstalling langchain-core-0.3.0:
      Successfully uninstalled langchain-core-0.3.0



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement langchain-text-aplitters==0.3.0 (from versions: none)
ERROR: No matching distribution found for langchain-text-aplitters==0.3.0

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Chat model

### 通常のチャット

In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [4]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [5]:
messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います！"),
    AIMessage("こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？"),
    HumanMessage("私の名前が分かりますか？"),
]

In [6]:
ai_message = model.invoke(messages)
print(ai_message.content)

はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいことがありますか？


### ストリーミング応答

In [7]:
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)
    time.sleep(0.1) # 本来不要だが、挙動確認のために追加

はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいことがありますか？

## ChatPromptTemplate

In [8]:
from langchain_core.prompts import ChatPromptTemplate

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザが入力した料理のレシピを教えてください。"),
        ("human", "{dish}"),
    ]
)       

In [10]:
prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

messages=[SystemMessage(content='ユーザが入力した料理のレシピを教えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [11]:
ai_message = model.invoke(prompt_value)
print(ai_message.content)

カレーの基本的なレシピをご紹介します。以下は、一般的なチキンカレーの作り方です。

### 材料
- 鶏肉（もも肉または胸肉）: 500g
- 玉ねぎ: 2個
- にんにく: 2片
- 生姜: 1片
- トマト: 1個（またはトマト缶）
- カレーパウダー: 大さじ2
- 塩: 適量
- 胡椒: 適量
- サラダ油: 大さじ2
- 水: 500ml
- お好みで野菜（じゃがいも、にんじん、ピーマンなど）

### 作り方
1. **下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒を振っておきます。
   - 玉ねぎは薄切り、にんにくと生姜はみじん切りにします。
   - トマトはざく切りにします。

2. **炒める**:
   - 鍋にサラダ油を熱し、玉ねぎを加えて中火で炒めます。玉ねぎが透明になるまで炒めます。
   - にんにくと生姜を加え、香りが立つまでさらに炒めます。

3. **鶏肉を加える**:
   - 鶏肉を鍋に加え、表面が白くなるまで炒めます。

4. **スパイスを加える**:
   - カレーパウダーを加え、全体に絡めるように炒めます。

5. **トマトと水を加える**:
   - トマトを加え、全体を混ぜたら、水を加えます。沸騰したら、アクを取り除きます。

6. **煮込む**:
   - 蓋をして弱火で約20〜30分煮込みます。お好みで野菜を加える場合は、このタイミングで入れます。

7. **味を調える**:
   - 最後に塩で味を調整し、必要に応じてさらに煮込んでください。

8. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みでパセリやヨーグルトを添えて完成です。

### お好みで
- 辛さを調整したい場合は、チリパウダーやカイエンペッパーを加えてください。
- ココナッツミルクを加えると、まろやかな味わいになります。

ぜひお試しください！


## OutPutParser

### PydanticOutputParser

In [12]:
from pydantic import BaseModel, Field

In [13]:
# 得たい出力形式をPydanticで定義
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="料理の材料")
    steps: list[str] = Field(description="料理の手順")

In [14]:
from langchain_core.output_parsers import PydanticOutputParser

In [15]:
# 定義したClassからPydanticOutputParserを作成
output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [16]:
# 作成したPydanticOutputParserに準拠するように指示するプロンプト
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "料理の材料", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "料理の手順", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザが入力した料理のレシピを教えてください。\n\n"
            "{format_instructions}",
        ),
        (
            "human",
            "{dish}",
        )
    ]
)

In [18]:
prompt_value = prompt.invoke(
    {
        "format_instructions": format_instructions,
        "dish": "カレー",
    }
)
print(prompt_value)

messages=[SystemMessage(content='ユーザが入力した料理のレシピを教えてください。\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"ingredients": {"description": "料理の材料", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "料理の手順", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}\n```', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [19]:
ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
    "塩 適量",
    "こしょう 適量"
  ],
  "steps": [
    "鶏肉は一口大に切り、塩とこしょうをふる。",
    "玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。",
    "鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、全体をよく混ぜる。",
    "水を加え、沸騰したらアクを取り、弱火で20分煮る。",
    "カレールーを加え、よく溶かしてさらに10分煮込む。",
    "味を見て、必要に応じて塩で調整する。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}


In [20]:
# 出力からインスタンスを作成
recipe = output_parser.invoke(ai_message)
recipe

Recipe(ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'], steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体をよく混ぜる。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。'])

### StrOutputParser

In [21]:
from langchain_core.output_parsers import StrOutputParser

In [22]:
output_parser = StrOutputParser()

In [23]:
ai_message = AIMessage(content="こんにちは、私はAIアシスタントです。")
output = output_parser.invoke(ai_message)
print(output)

こんにちは、私はAIアシスタントです。


## LCELによるChain

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザが入力した料理のレシピを教えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [25]:
chain = prompt | model | StrOutputParser()

In [26]:
ai_message = chain.invoke({"dish": "カレー"})
print(ai_message)

カレーの基本的なレシピをご紹介します。以下は、一般的なチキンカレーの作り方です。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんにく: 2片
- 生姜: 1片
- トマト: 1個（またはトマト缶）
- カレーパウダー: 大さじ2
- 塩: 適量
- 胡椒: 適量
- サラダ油: 大さじ2
- 水: 400ml
- お好みで野菜（じゃがいも、にんじん、ピーマンなど）

### 作り方
1. **下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒を振っておきます。
   - 玉ねぎは薄切り、にんにくと生姜はみじん切りにします。
   - トマトはざく切りにします。

2. **炒める**:
   - 鍋にサラダ油を熱し、玉ねぎを加えて中火で炒めます。玉ねぎが透明になるまで炒めます。
   - にんにくと生姜を加え、香りが立つまでさらに炒めます。

3. **鶏肉を加える**:
   - 鶏肉を鍋に加え、表面が白くなるまで炒めます。

4. **スパイスを加える**:
   - カレーパウダーを加え、全体に絡めるように炒めます。

5. **トマトと水を加える**:
   - トマトを加え、全体を混ぜたら、水を加えます。沸騰したら、アクを取り除きます。

6. **煮込む**:
   - 蓋をして弱火で約20分煮込みます。お好みで、じゃがいもやにんじんなどの野菜をこのタイミングで加えると良いでしょう。

7. **味を調える**:
   - 最後に塩で味を調整し、必要に応じてさらに煮込んでください。

8. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みでパセリやヨーグルトを添えて完成です。

### お好みで
- ココナッツミルクを加えると、まろやかな味わいになります。
- 辛さを調整したい場合は、チリパウダーやハラペーニョを加えてください。

ぜひお試しください！


### RAG

### Document loader（データの読み込み）

In [27]:
from langchain_community.document_loaders import GitLoader

In [28]:
def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

# Langchainでは様々なDocument loaderが提供されているが、ここではGitLoaderを例として利用
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

In [29]:
raw_docs = loader.load()

In [30]:
# .mdxファイルの個数
print(len(raw_docs))

385


### Document transformer（データの変換）

#### チャンク分割

In [31]:
from langchain_text_splitters import CharacterTextSplitter

In [32]:
# チャンク分割の設定
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [33]:
# チャンク分割の実行
docs = text_splitter.split_documents(raw_docs)

Created a chunk of size 6803, which is longer than the specified 1000
Created a chunk of size 3302, which is longer than the specified 1000
Created a chunk of size 1851, which is longer than the specified 1000
Created a chunk of size 1639, which is longer than the specified 1000
Created a chunk of size 9269, which is longer than the specified 1000
Created a chunk of size 2579, which is longer than the specified 1000
Created a chunk of size 17814, which is longer than the specified 1000
Created a chunk of size 1700, which is longer than the specified 1000
Created a chunk of size 1135, which is longer than the specified 1000
Created a chunk of size 1126, which is longer than the specified 1000
Created a chunk of size 1098, which is longer than the specified 1000
Created a chunk of size 1433, which is longer than the specified 1000
Created a chunk of size 1300, which is longer than the specified 1000
Created a chunk of size 1166, which is longer than the specified 1000
Created a chunk of 

In [34]:
# チャンク分割された個数
print(len(docs))

1351


Document transformerとしては、HTMLをプレーンテキストに変換したり、ドキュメントを翻訳する機能などが存在する

### Embedding model（ベクトル変換）

In [35]:
from langchain_openai import OpenAIEmbeddings

In [36]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### Vector store（ベクトル保存）

In [37]:
from langchain_chroma import Chroma

In [38]:
db = Chroma.from_documents(docs, embeddings)

### Retriever（検索インタフェース）

In [39]:
retriever = db.as_retriever()

### LCELでRAGを実行

In [40]:
from langchain_core.runnables import RunnablePassthrough

In [41]:
prompt = ChatPromptTemplate.from_template(
    """
    以下の文脈だけを踏まえ、質問に回答してください。

    文脈: ###
    {context}
    ###

    質問: ###
    {question}
    ###
    """
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [42]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [43]:
ai_message = chain.invoke("AWSのS3からデータを読み込むためのDocument loaderはありますか。")
print(ai_message)

はい、AWSのS3からデータを読み込むためのDocument loaderとして、`S3DirectoryLoader`と`S3FileLoader`があります。これらを使用することで、S3のディレクトリやファイルからデータを読み込むことができます。
